In [1]:
import numpy as np
import random

from IPython.display import Image
from IPython.core.display import HTML 

In [2]:
def generate_sequence(alphabet, length):
    return [random.choice(alphabet) for i in range(length)]

C = 20
N1 = 35
N2 = 37
M1 = 33
M2 = 41
ALPHABET = ['A', 'B', 'C', 'D']

# HW 2

In [3]:
def restore_ans(first, second, last, i, j):
    if i or j:
        i_last, j_last = last[i][j]
        ans_first, ans_second = restore_ans(first, second, last, i_last, j_last)
        if i == i_last:
            ans_first += '-'
        else:
            ans_first += first[i - 1]
        if j == j_last:
            ans_second += '-'
        else:
            ans_second += second[j - 1]
            
        return ans_first, ans_second
    else:
        return '', ''

def alignment(first, second, mu, delta):
    n = len(first)
    m = len(second)
    dp = np.ones((n + 1, m + 1)) * (-np.inf)
    last = np.empty((n + 1, m + 1), dtype=tuple)
    
    dp[0][0] = 0
    
    for i in range(n + 1):
        for j in range(m + 1):
            if i and j and first[i - 1] == second[j - 1] and dp[i][j] < dp[i - 1, j - 1] + 1:
                dp[i][j] = dp[i - 1, j - 1] + 1
                last[i][j] = (i - 1, j - 1)
            
            if i and j and dp[i][j] < dp[i - 1][j - 1] + delta:
                dp[i][j] = dp[i - 1][j - 1] + delta
                last[i][j] = (i - 1, j - 1)
                
            if i and dp[i][j] < dp[i - 1][j] + mu:
                dp[i][j] = dp[i - 1][j] + mu
                last[i][j] = (i - 1, j)
                
            if j and dp[i][j] < dp[i][j - 1] + mu:
                dp[i][j] = dp[i][j - 1] + mu
                last[i][j] = (i, j - 1)
                
    return dp[n][m], restore_ans(first, second, last, n, m)

MU = -0.9
DELTA = -0.6

common = generate_sequence(ALPHABET, C)
first = generate_sequence(ALPHABET, N1) + common + generate_sequence(ALPHABET, N2)
second = generate_sequence(ALPHABET, M1) + common + generate_sequence(ALPHABET, M2)

value, (first_ans, second_ans) = alignment(first, second, MU, DELTA)

print(first_ans)
print(second_ans)

equals = 0
gaps = 0
mismatches = 0
for i in range(len(first_ans)):
    if first_ans[i] == second_ans[i]:
        equals += 1
    elif first_ans[i] == '-' or second_ans[i] == '-':
        gaps += 1
    else:
        mismatches += 1
        
print('Equals = {}, gaps = {}. mismatches = {}'.format(equals, gaps, mismatches))
print('Expected score = {}, real score = {}'.format(value, equals + gaps * MU + mismatches * DELTA))

--A-B--CAA-DBD-CACBCCCACDAACDADDDBAAACBCCDCDDBDCBBAACDBDDDCACCDDBAC-ABADCCDCDABADADABBD-ACBB-ACACCD--AD---D
DDCBBBDDAACDBDABACADCCACDAA-D-CDD----C-CC--DDBDCBBAACDBDDDCACC-DCACDAB-DDCBADACACCCA-BDAACBBDA-ACCDBBADBABB
Equals = 64, gaps = 28. mismatches = 15
Expected score = 29.800000000000004, real score = 29.799999999999997


# HW 3

In [4]:
def alignment_with_matrix(first, second, matr):
    n = len(first)
    m = len(second)
    dp = np.ones((n + 1, m + 1)) * (-np.inf)
    last = np.empty((n + 1, m + 1), dtype=tuple)
    
    dp[0][0] = 0
    
    for i in range(n + 1):
        if i:
            f = first[i - 1]
        for j in range(m + 1):
            if j:
                s = second[j - 1]
                
            if i and j and dp[i][j] < dp[i - 1][j - 1] + matr[f, s]:
                dp[i][j] = dp[i - 1, j - 1] + matr[f, s]
                last[i][j] = (i - 1, j - 1)
            
            if i and dp[i][j] < dp[i - 1][j] + matr[f, '-']:
                dp[i][j] = dp[i - 1][j] + matr[f, '-']
                last[i][j] = (i - 1, j)
                
            if j and dp[i][j] < dp[i][j - 1] + matr[s, '-']:
                dp[i][j] = dp[i][j - 1] + matr[s, '-']
                last[i][j] = (i, j - 1)
                
    return dp[n][m], restore_ans(first, second, last, n, m)

common = generate_sequence(ALPHABET, C)
first = generate_sequence(ALPHABET, N1) + common + generate_sequence(ALPHABET, N2)
second = generate_sequence(ALPHABET, M1) + common + generate_sequence(ALPHABET, M2)

matr = {}
for i in ALPHABET:
    for j in ALPHABET:
        if (i, j) not in matr: 
            matr[i, j] = matr[j, i] = random.randint(5, 10) if i == j else random.randint(-5, 0) 
    matr[i, '-'] = matr['-', i] = random.randint(-5, 0) 

for i in ALPHABET + ['-']:
    for j in ALPHABET:
        print('{:3} '.format(matr[i, j]), end='')
    print()

value, (first_ans, second_ans) = alignment_with_matrix(first, second, matr)

print(first_ans)
print(second_ans)

equals = 0
gaps = 0
mismatches = 0
real_score = 0
for i in range(len(first_ans)):
    if first_ans[i] == second_ans[i]:
        equals += 1
    elif first_ans[i] == '-' or second_ans[i] == '-':
        gaps += 1
    else:
        mismatches += 1
    real_score += matr[first_ans[i], second_ans[i]]
    
print('Equals = {}, gaps = {}. mismatches = {}'.format(equals, gaps, mismatches))
print('Expected score = {}, real score = {}'.format(value, real_score))

  8  -5  -3  -3 
 -5  10  -4  -1 
 -3  -4   6  -3 
 -3  -1  -3   8 
 -2  -2  -5  -2 
---BDCBADDBBDDDCDBDBDCA-CBD-CABC-CCDAABCDBACDACCDACBACDDDCABCCCD-A-CD----DABC--ADDDBCCCACDCA-CBABCBCCDCAAC-A-
AAAB-CB---BBBACCC-CB-CABCBDACDBCAADDCA-C-BACDACCDACBACDDDCABCCBDBABBDBBABDABCBDABDD----A-DAABCB-B-BCCACACCDDB
Equals = 61, gaps = 32. mismatches = 16
Expected score = 349.0, real score = 349


# HW 4

In [5]:
def restore_local_ans(first, second, last, dp, i, j):
    if i or j:
        i_last, j_last = last[i][j]
        ans_first, ans_second = restore_local_ans(first, second, last, dp, i_last, j_last)
        
        if dp[i][j] == 0 and last[i][j] == (0, 0):
            return ans_first, ans_second
        
        if i == i_last:
            ans_first += '-'
        else:
            ans_first += first[i - 1]
        if j == j_last:
            ans_second += '-'
        else:
            ans_second += second[j - 1]
            
        return ans_first, ans_second
    else:
        return '', ''


def local_alignment(first, second, mu, delta):
    n = len(first)
    m = len(second)
    dp = np.ones((n + 1, m + 1)) * (-np.inf)
    last = np.empty((n + 1, m + 1), dtype=tuple)
    
    dp[0][0] = 0
    
    for i in range(n + 1):
        for j in range(m + 1):
            dp[i][j], last[i][j] = 0, (0, 0)
            
            if i and j and first[i - 1] == second[j - 1] and dp[i][j] < dp[i - 1, j - 1] + 1:
                dp[i][j] = dp[i - 1, j - 1] + 1
                last[i][j] = (i - 1, j - 1)
            
            if i and j and dp[i][j] < dp[i - 1][j - 1] + delta:
                dp[i][j] = dp[i - 1][j - 1] + delta
                last[i][j] = (i - 1, j - 1)
                
            if i and dp[i][j] < dp[i - 1][j] + mu:
                dp[i][j] = dp[i - 1][j] + mu
                last[i][j] = (i - 1, j)
                
            if j and dp[i][j] < dp[i][j - 1] + mu:
                dp[i][j] = dp[i][j - 1] + mu
                last[i][j] = (i, j - 1)
                
    best_value = -np.inf
    for i in range(n + 1):
        for j in range(m + 1):
            if dp[i][j] > best_value:
                best_value = dp[i][j]
                x, y = i, j
                
    return dp[x][y], restore_local_ans(first, second, last, dp, x, y)

MU = -1.9
DELTA = -1.3

common = generate_sequence(ALPHABET, C)
first  = generate_sequence(ALPHABET, N1) + common + generate_sequence(ALPHABET, N2)
second = generate_sequence(ALPHABET, M1) + common + generate_sequence(ALPHABET, M2)

value, (first_ans, second_ans) = local_alignment(first, second, MU, DELTA)

print(first_ans)
print(second_ans)


equals = 0
gaps = 0
mismatches = 0
for i in range(len(first_ans)):
    if first_ans[i] == second_ans[i]:
        equals += 1
    elif first_ans[i] == '-' or second_ans[i] == '-':
        gaps += 1
    else:
        mismatches += 1
        
print('Equals = {}, gaps = {}. mismatches = {}'.format(equals, gaps, mismatches))
print('Expected score = {}, real score = {}'.format(value, equals + gaps * MU + mismatches * DELTA))

DBCCDDCCADDDBDDDCADD
DBCCDDCCADDDBDDDCADD
Equals = 20, gaps = 0. mismatches = 0
Expected score = 20.0, real score = 20.0


# HW 5

In [6]:
def restore_ans(first, second, last, pos):
    if not last[pos] is None:
        last_pos = last[pos]
        
        i, j = pos[:2]
        i_last, j_last = last_pos[:2] 
        
        ans_first, ans_second = restore_ans(first, second, last, last_pos)
        
        if i == i_last:
            ans_first += '-'
        else:
            ans_first += first[i - 1]
        if j == j_last:
            ans_second += '-'
        else:
            ans_second += second[j - 1]
            
        return ans_first, ans_second
    else:
        return '', ''
    
def update(dp, last, value, frm, to):
    if dp[to] < dp[frm] + value:
        dp[to] = dp[frm] + value
        last[to] = frm
    
def gaps_alignment(first, second, mu, sigma, delta):
    n = len(first)
    m = len(second)
    dp = np.ones((n + 1, m + 1, 2, 2)) * (-np.inf)
    last = np.empty((n + 1, m + 1, 2, 2), dtype=tuple)
    
    dp[0][0][0][0] = 0
    
    for i in range(n):
        for j in range(m):
            for is_f_gap in range(2):
                for is_s_gap in range(2):
                    if i < n and j < m and first[i] == second[j]:
                        update(dp, last, 1, (i, j, is_f_gap, is_s_gap), (i + 1, j + 1, 0, 0))

                    if i < n and j < m:
                        update(dp, last, delta, (i, j, is_f_gap, is_s_gap), (i + 1, j + 1, 0, 0))

                    if i < n:
                        update(dp, last, sigma if is_f_gap else mu, (i, j, is_f_gap, is_s_gap), (i + 1, j, 1, 0))

                    if j < m:
                        update(dp, last, sigma if is_s_gap else mu, (i, j, is_f_gap, is_s_gap), (i, j + 1, 0, 1))

    val = -np.inf
    for is_f_gap in range(2):
        for is_s_gap in range(2):
            if dp[n][m][is_f_gap][is_s_gap] > val:
                val = dp[n][m][is_f_gap][is_s_gap]
                x, y = is_f_gap, is_s_gap
                
    return val, restore_ans(first, second, last, (n, m, x, y))

MU = -1.9
SIGMA = -0.1
DELTA = -1.3

common = generate_sequence(ALPHABET, C)
first  = generate_sequence(ALPHABET, N1) + common + generate_sequence(ALPHABET, N2)
second = generate_sequence(ALPHABET, M1) + common + generate_sequence(ALPHABET, M2)

value, (first_ans, second_ans) = gaps_alignment(first, second, MU, SIGMA, DELTA)

print(first_ans)
print(second_ans)


equals = 0
gaps = 0
mismatches = 0

is_last_f_gap = False
is_last_s_gap = False

real_score = 0

for i in range(len(first_ans)):
    if first_ans[i] == '-':
        real_score += SIGMA if is_last_f_gap else MU
        is_last_f_gap = True
        gaps += 1
    else:
        is_last_f_gap = False
    
    if second_ans[i] == '-':
        real_score += SIGMA if is_last_s_gap else MU
        is_last_s_gap = True
        gaps += 1
    else:
        is_last_s_gap = False

    if first_ans[i] == second_ans[i]:
        equals += 1
        real_score += 1
    elif not is_last_f_gap and not is_last_s_gap:
        mismatches += 1
        real_score += DELTA
        
print('Equals = {}, gaps = {}. mismatches = {}'.format(equals, gaps, mismatches))
print('Expected score = {}, real score = {}'.format(value, real_score))

---DBBBDABAACABBAAACAADAADDACBBBBDC------------DBBCBDAACCADCCDBCBBDAAACAAAACA--BBDDCCACBCBDCBBA---CDAC----------CBBDCBBDC---C
CCCABCBD-----------------DDCCBCABDCBCCDDDDDDAAADBBCBDAACCADCCDBCBBD----AAAACADDABDDCAA-------BABBBCDACADABDDDCADCDBD---DCDBBA
Equals = 52, gaps = 64. mismatches = 9
Expected score = 15.899999999999984, real score = 15.899999999999984


# HW 6

In [7]:
# print(''.join(generate_sequence(['A', 'C', 'G', 'T'], 100)))
# print(''.join(generate_sequence(['A', 'C', 'G', 'T'], 1000)))
# print(''.join(generate_sequence(['A', 'C', 'G', 'T'], 10000)))

In [8]:
Image(url='./100.png')

In [9]:
Image(url='./1000.png')

In [10]:
Image(url='./10000.png')